In [2]:
from tqdm.auto import tqdm
import itertools
import random
import pickle as pkl
import pandas as pd

## Load Dutch UMLS

In [27]:
df = pd.read_csv('NLWB.csv')

In [18]:
df = df.drop_duplicates(subset='mention', keep='first')

In [19]:
df

,Unnamed: 0,id,mention,cui,start_index,end_index,sentence,uniq_cui
0,0,0,aminozuren,['C0002520'],57,67,Uit simpele organische stoffen ontstonden mate...,C0002520
1,1,1,kopersulfaat,['C0056301'],52,64,Onder strikte voorwaarden is een bladbespuitin...,C0056301
2,2,2,kopen,['C0520949'],171,176,Individuele armoede kan het gevolg zijn van ee...,C0520949
3,3,3,ziekte van Van Leeuwenhoek,"['C0027066', 'C1854302']",168,194,Volgens de verhalen dicteerde hij op zijn ster...,C0027066
4,4,4,retina,['C0035298'],16,22,Het netvlies of retina is beweeglijk zodat het...,C0035298
...,...,...,...,...,...,...,...,...
7870,7870,8346,neutrofielen,['C0027950'],16,28,Het zwermen van neutrofielen is een type gecoö...,C0027950
7871,7871,8347,hals,['C3665420'],3,7,"De hals- en ruggenwervels dragen hoge, vertica...",C3665420
7873,7873,8349,spaanplaatgas,['C0016564'],37,50,Vanwege jarenlange blootstelling aan spaanplaa...,C0016564
7874,7874,8350,Hartcatheterisatie,['C0085532'],77,95,Chamuleaus enthousiasme voor de cardiologie we...,C0085532


In [20]:
cleaned = []

for index, concept in df.iterrows():
    cleaned.append(f"{concept['uniq_cui']}||{concept['mention']}")

In [21]:
cleaned[:30]

['C0002520||aminozuren',
 'C0056301||kopersulfaat',
 'C0520949||kopen',
 'C0027066||ziekte van Van Leeuwenhoek',
 'C0035298||retina',
 'C0016330||fluor',
 'C0038702||sulfonamide',
 'C1690571||allergische',
 'C5203670||COVID-19',
 'C0679560||analyse van DNA',
 'C0038984||zweet',
 "C3203533||trauma's",
 'C0007012||kooldioxide',
 'C0036471||balzak',
 'C0242350||impotent',
 'C0597101||basen',
 'C0016658||botzetting',
 'C0029850||klaplong',
 'C0013604||waterzucht',
 'C0026820||samen te trekken',
 'C4045985||wierp duivelen uit',
 'C0155020||blind',
 'C0021345||ziekte van Pfeiffer',
 'C0023343||leprozen',
 'C0007012||CO',
 'C0014499||epidemische ziekten',
 'C0006826||cancereuze aandoening',
 'C0567415||atomen',
 'C0022568||hoornvliesontsteking',
 'C0018524||hallucinogenen']

## positive pairs generation

In [22]:
umls_dict = {} # constrauct cui to list of name dict, again
for line in tqdm(cleaned):
    cui, name = line.split("||")
    if cui in umls_dict:
        umls_dict[cui].append(name)
    else:
        umls_dict[cui] = [name]

  0%|          | 0/2606 [00:00<?, ?it/s]

### generate!

In [23]:
def gen_pairs(input_list):
    return list(itertools.combinations(input_list, r=2))

In [24]:
pos_pairs = []
for k,v in tqdm(umls_dict.items()):
    pairs = gen_pairs(v)
    if len(pairs)>50: # if >50 pairs, then trim to 50 pairs
        pairs = random.sample(pairs, 50)
    for p in pairs:
        line = str(k) + "||" + p[0] + "||" + p[1]
        pos_pairs.append(line)

  0%|          | 0/999 [00:00<?, ?it/s]

In [25]:
pos_pairs[:100]

['C0002520||aminozuren||γ-aminozuur',
 'C0002520||aminozuren||α-aminozuren',
 'C0002520||aminozuren||α-aminozuur',
 'C0002520||γ-aminozuur||α-aminozuren',
 'C0002520||γ-aminozuur||α-aminozuur',
 'C0002520||α-aminozuren||α-aminozuur',
 'C0520949||kopen||koopverslaving',
 'C0520949||kopen||winkelverslaving',
 'C0520949||koopverslaving||winkelverslaving',
 'C0027066||ziekte van Van Leeuwenhoek||myoclonische',
 'C0016330||fluor||fluormolecuul',
 'C0016330||fluor||fluorgas',
 'C0016330||fluor||F',
 'C0016330||fluor||moleculair fluor',
 'C0016330||fluormolecuul||fluorgas',
 'C0016330||fluormolecuul||F',
 'C0016330||fluormolecuul||moleculair fluor',
 'C0016330||fluorgas||F',
 'C0016330||fluorgas||moleculair fluor',
 'C0016330||F||moleculair fluor',
 'C0038702||sulfonamide||sulfa-medicijnen',
 'C1690571||overgevoeligheid||allergische aandoeningen',
 'C1690571||overgevoelig||kattenallergie',
 'C1690571||overgevoelig||overgevoeligheden',
 'C1690571||overgevoeligheid||kattenallergie',
 'C1690571|

### save the pairwise positive training file

In [26]:
with open('./training_file_NLWB_no_dup_pairwise_pair_th50.txt', 'w') as f:
    for line in pos_pairs:
        f.write("%s\n" % line)